In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing libraries

In [ ]:
import matplotlib.pyplot as plt
from math import pi
import seaborn as sns
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.model_selection import cross_val_score

**Importing the dataset**

In [ ]:
dataset = pd.read_csv("/kaggle/input/family-income-and-expenditure/Family Income and Expenditure.csv")

In [ ]:
dataset.head()

Data Pre-processing

In [ ]:
msno.matrix(dataset)

In [ ]:
dataset.info()

In [ ]:
null_data = dataset[dataset.isnull().any(axis=1)]
print(null_data.shape)

7536 rows with missing values

In [ ]:
print(dataset.isnull().sum())

Removing the unnecessary features

In [ ]:
dataset=dataset.drop(['Household Head Occupation','Household Head Class of Worker'],axis=1)

In [ ]:
dataset.info()

Feature Selection

In [ ]:
plt.figure(figsize=(12,10))
cor=dataset.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["Total Household Income"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.5]
cols1=relevant_features.index
cols1= np.delete(relevant_features.index,0)

cols1=np.delete(cols1,0)
cols1

In [ ]:
Feature Selection(Method2)

In [ ]:
k = 20
target = 'Total Household Income'
corrmat = dataset.corr()
cols = corrmat.nlargest(k, target)[target].index
f, ax = plt.subplots(figsize=(10, 7))
cm = np.corrcoef(dataset[cols].values.T)
sns.set(font_scale=1.25)
s = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)

print(type(cols.values))
cols = np.delete(cols,0)
print(cols)
s.set_title("Top 20 variables Most Correlated with Total Household Income", size=15)


In [ ]:
expenditures = [column for column in dataset.columns if 'Expenditure' in column]
expenditures1=cols1.values

In [ ]:
expenditures

In [ ]:
expenditures1

In [ ]:
X = dataset.loc[:, expenditures1]
y = dataset['Total Household Income']

Visualization of relationship between feature and target label

In [ ]:
plt.figure(figsize=(20, 20))
i = 1
for exp in cols1 :
    plt.subplot(6,3,i)
    sns.regplot(x=X[exp], y=y)
    i += 1

In [ ]:
plt.figure(figsize=(15, 25))
i = 1
for exp in cols1 :
    plt.subplot(6,3,i)
    sns.histplot(X[exp])
    i += 1

This tells that a large number of participants in the dataset spends about the same amount for expenses. 

Most peaks are below the bin with the most number of datapoints, which tells us there are values to the extreme right that are skewing the curve heavily.

While a lot of people agree on spending a certain range, there also exists a group that are spending at a maximum

Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, random_state = 0)

Applying the Decision Tree Regression Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor  
  

regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(xTrain,yTrain)

In [ ]:
regressor.score(xTest,yTest)

Applying the Random Forest Regression Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(verbose=True, n_jobs=-1, n_estimators=1000)
rfr.fit(xTrain, yTrain)

In [ ]:
rfr.score(xTest, yTest)

Test with new data

In [ ]:
test_data=pd.read_csv("../input/test-data/sample.csv")
test_data.head()

In [ ]:
predicted_income=rfr.predict(test_data)
predicted_income


In [ ]:

final_DF=pd.DataFrame({'Predicted Income':predicted_income})


final_DF.to_csv("Family_income.csv")
